In [1]:
import csv
import pandas as pd
import numpy as np

txt_file_tf_idf = r"TF_IDF.res"
csv_file_tf_idf = r"TF_IDF.csv"

data_tf_idf = pd.read_csv(txt_file_tf_idf, sep=" ", header = None, names=["Id query", "Q", "Id doc", "Ranking", "Similtud", "Modelo"], index_col=["Id query","Id doc"])
modelo = data_tf_idf['Modelo'].iloc[0]
data_tf_idf.drop(['Q','Modelo'], axis=1, inplace=True)
data_tf_idf.columns = ["Ranking - "+modelo, "Similtud - "+modelo]
data_tf_idf.head()

Ranking - TF_IDF  Similtud - TF_IDF
Id query Id doc                                     
1        122                    0           3.022447
         206                    1           2.964958
         576                    2           2.863274
         548                    3           2.811322
         549                    4           2.770214

In [2]:
txt_file_bm25 = r"BM25.res"
csv_file_bm25 = r"BM25.csv"

data_bm25 = pd.read_csv(txt_file_bm25, sep=" ", names=["Id query", "Q", "Id doc", "Ranking", "Similtud", "Modelo"], index_col=["Id query","Id doc"])
modelo = data_bm25['Modelo'].iloc[0]
data_bm25.drop(['Q','Modelo'], axis=1, inplace=True)
data_bm25.columns = ["Ranking - "+modelo, "Similtud - "+modelo]
data_bm25.head()

Ranking - BM25b0.75  Similtud - BM25b0.75
Id query Id doc                                           
1        122                       0              4.428265
         206                       1              4.344036
         576                       2              4.195057
         548                       3              4.118941
         549                       4              4.058712

In [3]:
df = pd.concat([data_tf_idf, data_bm25],axis=1,join="outer")

# Ordeno por iDQUERY y luego por ranking del primero de los dos modelos
df = df.reset_index()
df = df.sort_values(["Id query",df.columns[2]])
df = df.set_index(["Id query", "Id doc"])
df.head()

df.to_csv("resultados_queries.csv")

In [4]:
# Elimino columnas que contienen valor de similtud (no me interesa para clcular spearman)
spearman = df.drop(df.columns[[1,3]], axis=1)
spearman.head(15)

Ranking - TF_IDF  Ranking - BM25b0.75
Id query Id doc                                       
1        122                    0                    0
         206                    1                    1
         576                    2                    2
         548                    3                    3
         549                    4                    4
         327                    5                    5
         351                    6                    6
         317                    7                    7
         91                     8                    8
         241                    9                    9
         569                   10                   10
         568                   11                   11
         449                   12                   12
         257                   13                   13
         461                   14                   14

In [5]:
def coeficienteSpearman(df):
    n = len(df)
    df['d'] = df[df.columns[0]] - df[df.columns[1]]
    df['d_cuadrado'] = np.power(df['d'],2)
    sum_d_cuadrado = sum(df['d_cuadrado']) 
    resultado = 1 - ( (6.0 * sum_d_cuadrado) / (n * ( np.power(n,2.0) - 1.0 ) ) )
    # WARNING: NO ANDA BIEN
    #if n > 20:
    #    resultado = resultado / np.sqrt( (1.0 - np.power( resultado,2.0 )) / (n - 2.0) )
    return resultado 


In [6]:
# Creo el data frame resultado en donde muestro el coeficiente obtenido para cada query
spearman = spearman.reset_index()
df = pd.DataFrame(index=np.arange(min(spearman['Id query']), max(spearman['Id query'])+1), columns=['Coeficiente (n=10)', 'Coeficiente (n=25)', 'Coeficiente (n=50)'], dtype = 'float64')
df.index.name = "Id query"

# A cada index le doy su coeficiente para los primeros 'n' resultados
for i in df.index:
    
    # creo dataframe 'query' con los datos del query con id 'i'
    query = spearman[spearman['Id query'] == i]
    query = query.drop("Id query", axis=1)
    query = query.set_index(["Id doc"])
    
    # Calculo coeficiente
    df.loc[i,"Coeficiente (n=10)"] = coeficienteSpearman(query.head(n=10)) 
    df.loc[i,"Coeficiente (n=25)"] = coeficienteSpearman(query.head(n=25)) 
    df.loc[i,"Coeficiente (n=50)"] = coeficienteSpearman(query.head(n=50)) 

# Restauro index 
spearman = spearman.set_index(["Id query", "Id doc"])

df.head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Coeficiente (n=10),Coeficiente (n=25),Coeficiente (n=50)
Id query,,,
1,1.000000,1.000000,1.000000
2,-1.484848,0.337308,0.730516
3,1.000000,1.000000,0.997983
4,1.000000,1.000000,1.000000
5,1.000000,1.000000,1.000000


In [7]:
df.to_csv("coeficiente_spearman.csv")